<a href="https://colab.research.google.com/github/jioffe502/kmerkounters/blob/main/kmercount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install mrjob
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ioffe.pem to ioffe.pem
User uploaded file "ioffe.pem" with length 1674 bytes


In [ ]:
from google.colab import files

# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving mrjob.conf to mrjob.conf
User uploaded file "mrjob.conf" with length 651 bytes


In [ ]:
!chmod 400 ioffe.pem

In [ ]:
%%file kmercount.py
from mrjob.job import MRJob
import json
import re


'''
Count the number of kmers. Base MR Job - No improvements
'''

class KmerCount(MRJob):

    def mapper(self, _, line):
        # Parse the FASTA file and extract the sequence
        if line.startswith(">"):
            seq = ""
        else:
            seq = line.strip()
            # Count the number of kmers of size 1
            for i in range(len(seq)):
                kmer = seq[i:i+1]
                yield kmer, 1
    
    def reducer(self, kmer, counts):
        # Sum the counts for each kmer
        yield kmer, sum(counts)


if __name__ == '__main__':
    KmerCount.run()

Overwriting kmercount.py


In [ ]:
%%file kmercount.py
from mrjob.job import MRJob
import json
import re


'''
Count the number of kmers. MR job with Combiner and defining kmer length
'''

class KmerCount(MRJob):

    def configure_args(self):
        super(KmerCount, self).configure_args()
        self.add_passthru_arg('--kmer-length', type=int, default=1, help='Length of the kmer')

    def mapper(self, _, line):
        # Parse the FASTA file and extract the sequence
        if line.startswith(">"):
            seq = ""
        else:
            seq = line.strip()
            kmer_length = self.options.kmer_length
            # Count the number of kmers of specified size
            for i in range(len(seq) - kmer_length + 1):
                kmer = seq[i:i+kmer_length]
                yield kmer, 1

    
    def reducer(self, kmer, counts):
        # Sum the counts for each kmer
        yield kmer, sum(counts)
    
    combiner = reducer



if __name__ == '__main__':
    KmerCount.run()


Overwriting kmercount.py


In [ ]:
%%file kmercount.py
from mrjob.job import MRJob
import json
import re


'''
Count the number of kmers. Optimized String Slicing technique + Combiner
'''

class KmerCount(MRJob):

    def configure_args(self):
        super(KmerCount, self).configure_args()
        self.add_passthru_arg('--kmer-length', type=int, default=1, help='Length of the kmer')

    def mapper(self, _, line):
        # Parse the FASTA file and extract the sequence
        if line.startswith(">"):
            seq = ""
        else:
            seq = line.strip()
            kmer_length = self.options.kmer_length
            # Count the number of kmers of specified size
            for i in range(len(seq) - kmer_length + 1):
                if i == 0 or seq[i-1] != seq[i+kmer_length-1]:
                    kmer = seq[i:i+kmer_length]
                    yield kmer, 1
                else:
                    yield kmer, 1

    def reducer(self, kmer, counts):
        # Sum the counts for each kmer
        yield kmer, sum(counts)

    combiner = reducer


if __name__ == '__main__':
    KmerCount.run()


Overwriting kmercount.py


In [ ]:
'''
Define mapper method with input parameters self, and line:
a. Check if the line starts with '>'
i. If True, set seq as an empty string
ii. If False, perform the following steps:
    1) Set seq as the line with whitespaces stripped
    2) Set kmer_length as the value of the kmer-length option
    3) Iterate through seq, from index 0 to (len(seq) - kmer_length + 1)
        a) Slice seq from i to (i+kmer_length) and assign it to kmer
        b) Yield kmer, 1

Define reducer method with input parameters self, kmer, and counts:
b. Yield kmer, sum of counts

c. Set combiner as the reducer method
'''

In [ ]:
'''
Define mapper method with input parameters self, and line:
a. Check if the line starts with '>'
i.  If True, set seq as an empty string
ii. If False, perform the following steps:
    1) Set seq as the line with whitespaces stripped
    2) Set kmer_length as the value of the kmer-length option
    3) Iterate through seq, from index 0 to (len(seq) - kmer_length + 1)
      a) If index i is 0 or the character at (i-1) is not equal to the character at (i+kmer_length-1)
          i. Slice seq from i to (i+kmer_length) and assign it to kmer
          ii. Yield kmer, 1
      b) If characters are equal, yield kmer, 1

Define reducer method with input parameters self, kmer, and counts:
b. Yield kmer, sum of counts

c. Set combiner as the reducer method
```



In [ ]:
%%time
!python kmercount.py --kmer-length 3 -r local /content/NA24385_illumina_hg19_chr1_first1000.fa > kmercount2.out

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/kmercount.root.20230427.221136.722768
Running step 1 of 1...
job output is in /tmp/kmercount.root.20230427.221136.722768/output
Streaming final output from /tmp/kmercount.root.20230427.221136.722768/output...
Removing temp directory /tmp/kmercount.root.20230427.221136.722768...
CPU times: user 33.6 ms, sys: 8.62 ms, total: 42.2 ms
Wall time: 3.43 s


In [ ]:
#run the program. Capture output
%%time
!python kmercount.py -r emr s3://kmer-finalproject/kmer/final.fa --cloud-tmp-dir=s3://kmer-finalproject/tmp --cluster-id=j-Q0LQJG4HUVI4 --conf-path /content/mrjob.conf > kmercountfinal.out

Creating temp directory /tmp/kmercount.root.20230427.221749.416619
uploading working dir files to s3://kmer-finalproject/tmp/kmercount.root.20230427.221749.416619/files/wd...
Copying other local files to s3://kmer-finalproject/tmp/kmercount.root.20230427.221749.416619/files/
Adding our job to existing cluster j-Q0LQJG4HUVI4
  master node is ec2-44-205-247-42.compute-1.amazonaws.com
Waiting for Step 1 of 1 (s-3RRAGFSZJUD8W) to complete...
  RUNNING for 0:00:27
     5.0% complete
  RUNNING for 0:01:01
     5.6% complete
  RUNNING for 0:01:35
     6.1% complete
  RUNNING for 0:02:08
     6.5% complete
  RUNNING for 0:02:42
     7.0% complete
  RUNNING for 0:03:15
     7.2% complete
  RUNNING for 0:03:48
     7.5% complete
  RUNNING for 0:04:22
     8.1% complete
  RUNNING for 0:04:55
     8.5% complete
  RUNNING for 0:05:29
     8.9% complete
  RUNNING for 0:06:02
     9.1% complete
  RUNNING for 0:06:35
     9.5% complete
  RUNNING for 0:07:09
    10.1% complete
  RUNNING for 0:07:42
   